### Metropolis-Hastings Implementation

$$y_i | \mu \sim N(\mu, 1) $$
$$ \mu \sim t(0, 1, 1) $$

In [11]:
# http://www.mit.edu/~ilkery/papers/MetropolisHastingsSampling.pdf

In [12]:
lg = function(mu, n , y_bar){
    mu2 = mu^2
    n * (y_bar*mu - mu2 / 2.0) - log(1.0 + mu2)
}

In [13]:
mh = function(n, ybar, n_iter, mu_init, cand_sd){
    mu_out = numeric(n_iter)
    accpt = 0
    
    # Step 1, setting the inital value for theta_not
    mu_now = mu_init
    lg_now = lg(mu=mu_now, n=n, y_bar=y_bar)
    
    for (i in 1:n_iter){
        
    # Step 2, Draw candidate from proposal distribution.
        # Set canidate distribution to a Normal Distribution.
        mu_cand = rnorm(1, mean=mu_now, sd=cand_sd)
        
        # Compute acceptance ratio, alpha.
        lg_cand = lg(mu=mu_cand, n=n, y_bar=y_bar)
        lalpha = lg_cand - lg_now
        alpha = exp(lalpha)
        
    # Step 3, Update parameters.
        unif = runif(1)
        if (unif < alpha){
            mu_now = mu_cand
            accpt = accpt + 1
            lg_now = lg_cand
        } # End of update
        
        mu_out[i] = mu_now
    
    } # End of loop
    
    list(mu=mu_out, accpt=accpt/n_iter)
} # End of function

In [14]:
 y = c(1.2, 1.4, -0.5, 0.3, 0.9, 2.3, 1.0, 0.1, 1.3, 1.9)

In [15]:
# Need to calculate y_bar

y_bar = mean(y)
n = length(y)

In [16]:
hist(y, freq=FALSE, xlim=c(-1, 3))
points(y, rep(0.0, n))
points(y_bar, 0.0, pch=19)
curve(dt(x, df=1), lty=2, add=TRUE)

In [17]:
set.seed(43)
post = mh(n=n, ybar=y_bar, n_iter=1e3, mu_init=30.0, cand_sd=0.9)
str(post)

List of 2
 $ mu   : num [1:1000] 30 30 30 30 30 ...
 $ accpt: num 0.393


In [18]:
library("coda")
traceplot(as.mcmc(post$mu))

In [19]:
post$mu_keep = post$mu[-c(1:100)]
plot(density(post$mu_keep), xlim=c(-1.0, 3.0))
points(y_bar, 0.0, pch=19)
curve(dt(x, df=1),lty=2, add=TRUE)

In [20]:
# Need to know if the Markov chain is stationary.